In [1]:
import pytest
import torch
import vcr
import yaml
from api.binance import BinanceAPI
from helpers import prepare_data
from datastructures import TickerData, TickerStream
import requests
from ml.models.BSM import BSM4

with open("config.yaml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)


In [2]:
c = cfg["models"]["resnet"]["symbols"]

In [3]:
def get_ticker_data( symbol: str, interval: str = "1m", limit: int = 1000
    ):
        parameters = {
            "symbol": symbol,
            "interval": interval,
            "limit": limit,
            'startTime': 1639718100000
        }
        print(symbol)
        response = requests.get("https://api.binance.com" + "/api/v3/klines", params=parameters)
        print(response.status_code)

        tickers = [TickerData(symbol, interval, *x) for x in response.json()]
        stream = TickerStream(tickers, symbol, interval)
        last_timestamp = stream.max_timestamp()

        return stream

In [72]:
results = {s: get_ticker_data(symbol = s) for s in c}

DOGEUSDT
200
AVAXUSDT
200
SOLUSDT
200
SHIBUSDT
200
EURUSDT
200
GBPUSDT
200
ETCETH
200
ETCBTC
200
MKRUSDT
200
MKRBTC
200
IOTAUSDT
200
ADAUSDT
200
XLMUSDT
200
TRXUSDT
200
XMRUSDT
200
EOSUSDT
200
DOGEGBP
200
BTCEUR
200
BTCGBP
200
BTCUSDT
200


In [73]:
data = prepare_data(cfg, results)

In [74]:
data.shape

(180, 1000)

In [75]:
net = BSM4(n_features=180, n_outputs=20)

In [76]:
net.load_state_dict(torch.load('ml/models/BSM4.pth'))
net.eval()
net(torch.Tensor(data).unsqueeze(0)[:,:,:512])

tensor([[ 3.4461e-04, -5.6505e-04, -4.5657e-04, -3.0175e-06, -4.6906e-04,
         -5.2762e-05,  1.1928e-04,  5.7779e-05, -2.0853e-04, -1.7189e-04,
         -2.7889e-04,  5.4165e-04, -3.5790e-04,  1.4117e-04,  1.7991e-04,
         -1.2132e-03, -9.9808e-05, -4.9919e-06,  1.9482e-04,  4.7232e-04]],
       grad_fn=<AddmmBackward0>)

tensor([[ 3.4461e-04, -5.6505e-04, -4.5657e-04, -3.0175e-06, -4.6906e-04,
         -5.2762e-05,  1.1928e-04,  5.7779e-05, -2.0853e-04, -1.7189e-04,
         -2.7889e-04,  5.4165e-04, -3.5790e-04,  1.4117e-04,  1.7991e-04,
         -1.2132e-03, -9.9808e-05, -4.9919e-06,  1.9482e-04,  4.7232e-04]],
       grad_fn=<AddmmBackward0>)

In [39]:
torch.Tensor(data).unsqueeze(0)[:,:,:512].shape

torch.Size([1, 180, 512])

In [11]:
data.shape

(180, 1000)

In [52]:
a[(torch.Tensor(data).unsqueeze(0)[:,:,:512].numpy()[0] == a)]

array([ 0.5417852 ,  0.543265  ,  0.5425251 , ..., -0.27446878,
       -0.40040955, -0.4040477 ], dtype=float32)

In [53]:
torch.Tensor(data).unsqueeze(0)[:,:,:512].numpy()[0][(torch.Tensor(data).unsqueeze(0)[:,:,:512].numpy()[0] == a)]

array([ 0.5417852 ,  0.543265  ,  0.5425251 , ..., -0.27446878,
       -0.40040955, -0.4040477 ], dtype=float32)

In [23]:
import numpy as np
a = np.fromfile('a.file')

In [27]:
import pickle
with open('a.file','rb') as file:
    a = pickle.load(file)

In [28]:
a.shape

(180, 512)

In [29]:
data.shape

(180, 1000)

In [36]:
(data[:,:512]==a).mean()

0.9998914930555556